In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

import numpy as np
import scipy.misc

In [21]:
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
    
def to_cuda(x):
    return x.to(device)


In [22]:
def one_hot_encoder(y, num_classes=10):
    one_hot = torch.LongTensor(y.size(0), num_classes).zero_()
    one_hot.scatter_(1,y,1)
    return one_hot

In [23]:
class Discriminator(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 1):
        
        super(Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 128, 3, stride = 2, padding = 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, 3, stride = 2, padding = 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,512, 3, stride = 2, padding = 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.AvgPool2d(4)
        )
        
        self.layer4 = nn.Sequential(
            nn.Linear(512,1),
            nn.Sigmoid()
        )
        
    def forward(self, x, y = None):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        
        return out

In [32]:
class Generator(nn.Module):
    """
        Convolutional Generator for MNIST
    """
    def __init__(self, input_size=100, label_size=10, num_classes=784):
        super(Generator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 4*4*512),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(# input: 4 by 4, output: 7 by 7
            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(# input: 7 by 7, output: 14 by 14
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )
        self.layer4 = nn.Sequential(# input: 14 by 14, output: 28 by 28
            nn.ConvTranspose2d(128, 1, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        
    def forward(self, x, y=None):
        x = x.view(x.size(0), -1)
        y_ = self.layer1(x)
        y_ = y_.view(y_.size(0), 512, 4, 4)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        y_ = self.layer4(y_)
        return y_

In [33]:
D = to_cuda(Discriminator())
G = to_cuda(Generator())

In [34]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))]
)

In [35]:
mnist = datasets.MNIST(root = 'MNIST-data/', train = True,
                       transform = transform, download = False)

In [36]:
batch_size = 128
condition_size = 10

data_loader = DataLoader(dataset=mnist, batch_size = batch_size,
                         shuffle = True, drop_last = True)
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr = 0.0002)
G_opt = torch.optim.Adam(G.parameters(), lr = 0.0002)

max_epoch = 100
step = 0
n_critic = 1
n_noise = 100

D_labels = to_cuda(torch.ones(batch_size)) # Discriminator Label to real
D_fakes = to_cuda(torch.zeros(batch_size)) # Discriminator Label to fake

In [42]:
for epoch in range(max_epoch):
    for idx, (images, labels) in enumerate(data_loader):
        step += 1
        # Training Discriminator
        x = to_cuda(images)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, D_labels)

        z = to_cuda(torch.randn(batch_size, n_noise))
        z_outputs = D(G(z))
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        if step % n_critic == 0:
            # Training Generator
            z = to_cuda(torch.randn(batch_size, n_noise))
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_labels)

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.data[0], G_loss.data[0]))
                    
        if epoch % 5 == 0:
            G.eval()
            img = get_sample_image()
            scipy.misc.imsave('sample/{}_epoch_{}_type1.jpg'.format(MODEL_NAME, epoch), img)
            G.train()

/Users/krishna/.edm/envs/py3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/Users/krishna/.edm/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 2/100, Step: 1000, D Loss: 0.0351124070584774, G Loss: 4.231560230255127


KeyboardInterrupt: 

In [38]:
def get_sample_image(n_noise=100):
    """
        save sample 100 images
    """
    for num in range(10):
        for i in range(10):
            z = to_cuda(torch.randn(1, n_noise))
            y_hat = G(z)
            line_img = torch.cat((line_img, y_hat.view(28, 28)), dim=1) if i > 0 else y_hat.view(28, 28)
        all_img = torch.cat((all_img, line_img), dim=0) if num > 0 else line_img
    img = all_img.cpu().data.numpy()
    return img

In [40]:
MODEL_NAME = 'DCGAN'